# Cohort Analysis retention / churn Analysis with servival curve

In [1]:
## Libraries import

In [3]:
from scipy.optimize import minimize
#from autograd import value_and_grad, hessian
import numpy as np
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
output_notebook()
import holoviews as hv
from holoviews import opts
hv.extension('matplotlib')
%env HV_DOC_HTML=true
import hvplot.pandas
from matplotlib import pyplot as plt
import dateutil
from datetime import datetime
from datetime import datetime as dt
from pytz import utc
from datetime import datetime
import lifelines

Loading BokehJS ...

ModuleNotFoundError: No module named 'holoviews'

In [ ]:
d_parser = lambda x: pd.to_datetime(x, format='%Y-%m-%dT%H:%M:%S%z')

df = pd.read_csv('../data/raw/Tutorial3_Sample_Data_for_Cohort_Analysis_Survival_Curve_source_0.csv',\
                 parse_dates=['first_date','last_date'], date_parser=d_parser)

In [ ]:
df

In [ ]:
min = df['first_date'].min()
max = df['last_date'].max()

In [ ]:
# create duration from start_date and end  _date in different level of granulation
df['datediff'] =   ((pd.to_datetime(df['last_date'],utc=True) - pd.to_datetime(df['first_date'],utc=True))/pd.Timedelta(7, unit='D')).apply(np.ceil)
df.dropna(axis='rows', how='any', inplace=True)


In [ ]:
T=df['datediff']
#df.info()
E = df['canceled'].map({True: 1, False: 0})
# create duration from start_date and end  _date in different level of granulation
from lifelines.utils import datetimes_to_durations

In [ ]:
from lifelines import KaplanMeierFitter
kmf = KaplanMeierFitter(alpha=0.05) # calculate a 95% confidence interva
kmf.fit(durations=T, event_observed=E)
kmf.fit(T, E)

In [ ]:
# plot attributes
plt.figure(figsize=(12,8))
plt.title('All Subscriptions', fontsize='xx-large')

# y-axis
plt.ylabel('Survival Rate', fontsize='x-large')
plt.ylim((0.0, 1.0))
plt.yticks(np.arange(0.0,1.0,0.05))
plt.axhline(0.5, color='red', alpha=0.75, linestyle=':') # median line in red

# x-axis
plt.xlabel('Timeline (Days)', fontsize='x-large')
plt.xticks(range(0,800,30))
plt.axvline(30, color='gray', alpha=0.5, linestyle=':')  # 30-day gray dashed line
plt.axvline(180, color='gray', alpha=0.5, linestyle=':')  # 30-day gray dashed line
plt.axvline(365, color='gray', alpha=0.5, linestyle=':') # 1-year gray dashed line
plt.axvline(365*2, color='gray', alpha=0.5, linestyle=':') # 2-year gray dashed line

kmf.plot_survival_function()